# Natural Language Processing Course Project_Text/Data Summerization Application deployed on Flask  (Extractive and Abstractive)
###(Flask and UI code seperate)
# --------------------------------------------------------------------

## Submitted By : Muhammad Raamish Alam(20k-1326), Muhammad Ashar Ansari(20k-1409),  Muzammil Ahmed(20k-1394)

## Submitted To: Sir Raza Abbas


# --------------------------------------------------------------------
## Subject : NLP(DS 5007)

# Extractive Summarization:


#### Sumy to summarize text:
- Firstly do text data preprocessing (tokenization, plain text/html parsing)  
- Secondly instantiate a Sumy summarizer with a given algo. (e.g. LexRank), optional with stemmer and removing stopwords   
- Finally use the summarizer to extract a summary out of text 


### Using Sumy's LexRank

In [18]:
pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
#Import library essentials
from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser here, HTML parsers also available.as
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in.

#### Reference: 
#### https://gist.github.com/lastlegion/dd7f11aada4673dfbb4b
#### https://github.com/miso-belica/sumy/blob/dev/README.md
#### http://ai.intelligentonlinetools.com/ml/text-summarization/

#### Summarize a plain text file 


In [22]:
# Convert text to UTF-8
import os    
from chardet import detect

srcfile = "/content/sample_data/plain_text_sample_sec.txt"
trgfile = "/content/sample_data/plain_text_sample_sec_ENC.txt"

# get file encoding type
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

from_codec = get_encoding_type(srcfile)

# add try: except block for reliability
try: 
    with open(srcfile, 'r', encoding=from_codec) as f, open(trgfile, 'w', encoding='utf-8') as e:
        text = f.read() # for small files, for big use chunks
        e.write(text)

    os.remove(srcfile) # remove old encoding file
    os.rename(trgfile, srcfile) # rename new encoding
except UnicodeDecodeError:
    print('Decode Error')
except UnicodeEncodeError:
    print('Encode Error')

In [23]:
file = "/content/sample_data/plain_text_sample_sec.txt" #put the sample text file in the same folder as this notebook 
#Read the original text file in Python:
with open(file) as f:
    contents = f.read()
    print("===Original Text===\n"+contents)

===Original Text===
Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings. Robots are widely used in such industries as automobile manufacture to perform simple repetitive tasks, and in industries where work must be performed in environments hazardous to humans. Many aspects of robotics involve artificial intelligence; robots may be equipped with the equivalent of human senses such as vision, touch, and the ability to sense temperature. Some are even capable of simple decision making, and current robotics research is geared toward devising robots with a degree of self-sufficiency that will permit mobility and decision-making in an unstructured environment. Today’s industrial robots do not resemble human beings; a robot in human form is called an android. Compared to other artifacts with single functions, robots are expected to perform a variety of tasks, among which communication has an important role. A typical example is a hu

In [24]:
#which languages are supported: 
#https://stackoverflow.com/questions/15111183/what-languages-are-supported-for-nltk-word-tokenize-and-nltk-pos-tag
parser = PlaintextParser.from_file(file, Tokenizer("english"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, 3) #Summarize the document with 3 sentences, a tunable hyperparam 
    
print(f'\nSumy outcome type:{type(summary)}, element type:{type(summary[0])} \n{summary[:1]}')
allstr = '\n'.join(str(sentence) for sentence in summary)
print(f'\n===Extractive Summary using Sumy LexRank:===\n{allstr}')


Sumy outcome type:<class 'tuple'>, element type:<class 'sumy.models.dom._sentence.Sentence'> 
(<Sentence: Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings.>,)

===Extractive Summary using Sumy LexRank:===
Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings.
A typical example is a humanoid robot, the building of which has been regarded as the final target of robotics for many years, especially in Japan where many robotics researchers have been struggling with making humanoid robots work in our society.
Pfeifer and Scheier's book from 1999 entitled Understanding Intelligence shows a constructive approach to understanding intelligence with a variety of robots from the viewpoint of embodied cognitive science.


In [26]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

canidates = [contents.split()]
references = [allstr.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(contents, allstr)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:5.1304923275757076e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=0.177319587628866, fmeasure=0.3012259194395797), 'rougeL': Score(precision=1.0, recall=0.177319587628866, fmeasure=0.3012259194395797)}


#### Observation
- Command: summarizer(parser.document, SENTENCES_COUNT) where SENTENCES_COUNT is a tunable hyperparam
- Sumy outputs a tuple of objects of "sumy.models.dom._sentence.Sentence" class, that is the extracted summary sentences  
- We concatenate them with new lines to form a single text string to display the extrative summary produced by Sumy LexRank summerizer

#### Summarize a text string 

In [28]:
### Are you talking about multi-line strings? Easy, use triple quotes to start and end them.
string = "Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all. The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on the night when Mr. Dursley had seen that fateful news report about the owls. Only the photographs on the mantelpiece really showed how much time had passed. Ten years ago, there had been lots of pictures of what looked like a large pink beach ball wearing different-colored bonnets - but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy riding his first bicycle, on a carousel at the fair, playing a computer game with his father, being hugged and kissed by his mother. The room held no sign at all that another boy lived in the house, too."

parser = PlaintextParser.from_string(string, Tokenizer("english"))
summary = summarizer(parser.document, 2) #Summarize the string with 2 sentences

for sentence in summary:
    print(sentence)

Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all.
The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on the night when Mr. Dursley had seen that fateful news report about the owls.


In [30]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

allstr = '\n'.join(str(sentence) for sentence in summary)

canidates = [string.split()]
references = [allstr.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(string, allstr)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:5.0454056966289525e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=0.4647058823529412, fmeasure=0.6345381526104418), 'rougeL': Score(precision=1.0, recall=0.4647058823529412, fmeasure=0.6345381526104418)}


#### Try another language 

In [31]:
### Are you talking about multi-line strings? Easy, use triple quotes to start and end them.
string = """Près de dix ans s'étaient écoulés depuis que les Dursley s'étaient réveillés pour trouver leur neveu sur le perron, mais Privet Drive n'avait pratiquement pas changé. Le soleil se levait sur les mêmes jardins de devant bien rangés et éclairait le numéro quatre en laiton sur la porte d'entrée des Dursley ; elle se glissa dans leur salon, qui était presque exactement la même que la nuit où M. Dursley avait vu ce reportage fatidique sur les hiboux. Seules les photographies sur la cheminée montraient vraiment combien de temps s'était écoulé. Il y a dix ans, il y avait eu beaucoup de photos de ce qui ressemblait à un gros ballon de plage rose portant des bonnets de différentes couleurs - mais Dudley Dursley n'était plus un bébé, et maintenant les photos montraient un grand garçon blond conduisant son premier vélo, sur un carrousel à la foire, jouer à un jeu vidéo avec son père, se faire étreindre et embrasser par sa mère. La pièce ne contenait aucun signe qu'un autre garçon vivait également dans la maison."""

parser = PlaintextParser.from_string(string, Tokenizer("german"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, 2) #Summarize the string with 2 sentences

for sentence in summary:
    print(sentence)

Près de dix ans s'étaient écoulés depuis que les Dursley s'étaient réveillés pour trouver leur neveu sur le perron, mais Privet Drive n'avait pratiquement pas changé.
Le soleil se levait sur les mêmes jardins de devant bien rangés et éclairait le numéro quatre en laiton sur la porte d'entrée des Dursley ; elle se glissa dans leur salon, qui était presque exactement la même que la nuit où M. Dursley avait vu ce reportage fatidique sur les hiboux.


#### Summarize a HTML file

In [32]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
from sumy.parsers.html import HtmlParser #other parsers available for HTML etc.
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words


In [33]:
LANGUAGE = "english"
SENTENCES_COUNT = 5

url = "https://en.wikipedia.org/wiki/Combat"
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
# For plain text files:
# parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))

print('Test 1: simply using LeRank w/o stemming and not removing stop words:\n')
summarizer = LexRankSummarizer()
#stemmer = Stemmer(LANGUAGE)
#summarizer = LexRankSummarizer(stemmer)
#summarizer.stop_words = get_stop_words(LANGUAGE)

summary = summarizer(parser.document, SENTENCES_COUNT) #Summarize the document with pre-defined no. of sentences
for sentence in summary:
    print(sentence)

print('\nTest2: using LexRankSummarizer with stemming and removing stop words:\n')
#summarizer = LexRankSummarizer()
stemmer = Stemmer(LANGUAGE)
summarizer = LexRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

summary = summarizer(parser.document, SENTENCES_COUNT) #Summarize the document with pre-defined no. of sentences
for sentence in summary:
    print(sentence)

Test 1: simply using LeRank w/o stemming and not removing stop words:

Combat is sometimes resorted to as a method of self-defense, or can be used as a tool to impose one's will on others.
An instance of combat can be a stand-alone confrontation or a small part of a much larger violent conflict.
Instances of combat may also be benign and recreational, as in the cases of combat sports and mock combat.
Wikibooks has a book on the topic of: Fighting
Propensity to Serve and Motivation to Enlist among American Combat Soldiers."

Test2: using LexRankSummarizer with stemming and removing stop words:

Purposeful violent conflict
Combat( French for fight) is a purposeful violent conflict meant to physically harm or kill the opposition.
An instance of combat can be a stand-alone confrontation or a small part of a much larger violent conflict.
Combat Motivation in Today's Soldiers: U.S. Army War College Strategic Studies Institute."
Cohesion during Military Operations: A Field Study on Combat Uni

In [34]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

allstr = '\n'.join(str(sentence) for sentence in summary)
contents = '\n'.join(str(summary) for summary in summary)

canidates = [contents.split()]
references = [allstr.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(contents, allstr)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:8.444234747540162e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}


#### Observation
- Above I did two tests. It seems using Sumy LexRank summarizer with stemming and removing stop words gives a more concise summary, but less like haman writing
- You can try parsing & summarizing any (wiki) HTML page of your interest to see how it goes      

#### Sumy also contains command line utility for quick summarization of documents

In [ ]:
!sumy lex-rank --length=10 --url=https://en.wikipedia.org/wiki/Automatic_summarization # what's auto summarization?

Automatic summarization is the process of shortening a set of data computationally, to create a subset (a summary) that represents the most important or relevant information within the original content.
Abstractive summarization methods generate new text that did not exist in the original text.
An example of a summarization problem is document summarization, which attempts to automatically produce an abstract from a given document.
Another keyphrase extraction algorithm is TextRank.
Features might include the position in the document (i.e., the first few sentences are probably important), the number of words in the sentence, etc.
The main difficulty in supervised extractive summarization is that the known summaries must be manually created by extracting sentences so the sentences in an original training document can be labeled as "in summary" or "not in summary".
Automatic Text Summarization.
"Learning Algorithms for Keyphrase Extraction".
"A method for evaluating modern systems of aut

### Use Sumy's other summarizers (LSA, Luhn and Edmundson, etc.)

#### Sumy's summarization methods: https://miso-belica.github.io/sumy/summarizators.html

Edmundson added 3 more heuristics to the Luhn method to measure the importance of the sentences:   
`bonus words - positively relevant 
stigma words - negatively relevant 
null words - irrelevant words`

In [ ]:
# Summarize plain text file:
from sumy.summarizers.lsa import LsaSummarizer
parser = PlaintextParser.from_file(file, Tokenizer("english"))
summarizer = LsaSummarizer()
summary = summarizer(parser.document, 3) #Summarize the document with 3 sentences
for sentence in summary:
    print(sentence)

Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings.
A typical example is a humanoid robot, the building of which has been regarded as the final target of robotics for many years, especially in Japan where many robotics researchers have been struggling with making humanoid robots work in our society.
Although these issues have been attacked in the existing disciplines such as brain science, neuroscience, cognitive science, and developmental psychology, robotics may project a new light on understanding these issues by constructing artifacts similar to us.


In [ ]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

allstr = '\n'.join(str(sentence) for sentence in summary)

canidates = [contents.split()]
references = [allstr.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(contents, allstr)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:5.1304923275757076e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=0.19175257731958764, fmeasure=0.32179930795847755), 'rougeL': Score(precision=1.0, recall=0.19175257731958764, fmeasure=0.32179930795847755)}


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

#### A like-for-like comparison between LexRank, Luhn, Edmundson and LSA using the same test sample below: 

In [ ]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lsa import LsaSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer  
 
LANGUAGE = "english"
SENTENCES_COUNT = 5
 
# url = "https://en.wikipedia.org/wiki/Snorkeling"
url = "https://en.wikipedia.org/wiki/Combat"
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)

print ("--LexRankSummarizer--")    
summarizer = LexRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence1 in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence1)

print ("--LsaSummarizer--")    
summarizer = LsaSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence2 in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence2)
         
print ("--LuhnSummarizer--")   
summarizer = LuhnSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)
#summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this",)
for sentence3 in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence3)
         
print ("--EdmundsonSummarizer--")   
summarizer = EdmundsonSummarizer(stemmer) 
summarizer.stop_words = get_stop_words(LANGUAGE) 

summarizer.bonus_words = ("greatly","popular","recreational" )
summarizer.stigma_words = ("slowly","little")    
summarizer.null_words = ("another", "and", "or", "with", "be", "from")
for sentence4 in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence4)     

--LexRankSummarizer--
Purposeful violent conflict
Combat( French for fight) is a purposeful violent conflict meant to physically harm or kill the opposition.
An instance of combat can be a stand-alone confrontation or a small part of a much larger violent conflict.
Combat Motivation in Today's Soldiers: U.S. Army War College Strategic Studies Institute."
Cohesion during Military Operations: A Field Study on Combat Units in the Al-Aqsa Intifada."
--LsaSummarizer--
Combat( French for fight) is a purposeful violent conflict meant to physically harm or kill the opposition.
An instance of combat can be a stand-alone confrontation or a small part of a much larger violent conflict.
^ North Atlantic Treaty Organization, NATO Standardization Agency AAP-6 – Glossary of terms and definitions, p. 80
Martin van Creveld: The Changing Face of War: Lessons of Combat, from the Marne to Turkey.
Combat Motivation in Today's Soldiers: U.S. Army War College Strategic Studies Institute."
--LuhnSummarizer--


In [ ]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

allstr = '\n'.join(str(parser) for parser in summary)
contents = '\n'.join(str(sentence1) for sentence1 in summary)

canidates = [contents.split()]
references = [allstr.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(contents, allstr)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:7.720899511627474e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}


#### Observation
- LexRank seems give the most concice and coherent summary, closely followed by Edmundson module.

### Gensim
- Reference: http://ai.intelligentonlinetools.com/ml/text-summarization/
### Use Gensim's TextRank

#### Install Gensim:
"pip install gensim==3.8.3"

Note: if see the `No module named ‘gensim.summarization` error, refer to https://discuss.streamlit.io/t/no-module-named-gensim-summarization/11780 to resolve.  
To continue using gensim.summarization, you will have to downgrade the version of Gensim to gensim==3.8.3 or older.

In [ ]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import requests

In [ ]:
print ('Summary (a plain text file using Gensim):')
# Summarize plain text file stored locally
fname="/content/sample_data/plain_text_sample_sec.txt"
with open(fname, 'r') as myfile:
      text=myfile.read()     

SUMMARY_RATIO = 0.5 #for small file
# note: this is the ratio of summary text to the whole raw text, a tunable hypermater (do tune it to your specifc doc!)
#You should change it according to how big the raw text is. 
#e.g. if raw text is small(<=10 sentences), use ratio>=0.1; 
#if raw text is big (>100 sentences), use ratio<=0.01
KEYWORD_RATIO = 0.1 
sum = summarize(text, ratio=SUMMARY_RATIO)
print (sum)
print ('\nKeywords:')
print (keywords(text, ratio=KEYWORD_RATIO))

Summary (a plain text file using Gensim):
Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings.
Robots are widely used in such industries as automobile manufacture to perform simple repetitive tasks, and in industries where work must be performed in environments hazardous to humans.
Some are even capable of simple decision making, and current robotics research is geared toward devising robots with a degree of self-sufficiency that will permit mobility and decision-making in an unstructured environment.
Compared to other artifacts with single functions, robots are expected to perform a variety of tasks, among which communication has an important role.
From the viewpoint of biology, human beings can be discriminated from other species by three distinctive features or capabilities.
The first two capabilities have been attacked by robotics researchers as challenges in locomotion and manipulation that are the main issues in robotic

In [ ]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

# allstr = '\n'.join(str(parser) for parser in summary)
# contents = '\n'.join(str(sentence1) for sentence1 in summary)

canidates = [text.split()]
references = [sum.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(text, sum)
print(f'ROUGE SCORE:{scores}')

BLEU SCORE:5.1304923275757076e-232
ROUGE SCORE:{'rouge1': Score(precision=1.0, recall=0.5876288659793815, fmeasure=0.7402597402597402), 'rougeL': Score(precision=1.0, recall=0.5876288659793815, fmeasure=0.7402597402597402)}


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
print ('Summary (an online article using Gensim & web scraping using beautifulsoup):')   
#getting text document from web, below function based from 3
from bs4 import BeautifulSoup
from urllib.request import urlopen
#from urllib2 import urlopen #obselte Python2, The urllib2 module has been split across several modules in Python3 
#named urllib.request and urllib.error!

def get_only_text(url):
 """ 
     return the title and the text of the article at the specified url
 """
 page = urlopen(url)
 soup = BeautifulSoup(page, "lxml")
 text = ' '.join(map(lambda p: p.text, soup.find_all('p')))#Python map() iterates every element of an iterable 
 return soup.title.text, text    

# url = "https://en.wikipedia.org/wiki/Snorkeling"
url="https://en.wikipedia.org/wiki/Deep_learning"
title, text = get_only_text(url)
print(f'wiki article title is:{title}, Its text is:\n{text[:100]}...')
#print(' '.join(text))
SUMMARY_RATIO = 0.03 #a tunable hypermater 
KEYWORD_RATIO = 0.001
print('Wiki Summary:')
sum = summarize(title+text, ratio=SUMMARY_RATIO)
print (sum)
print ('\nKeywords:') 
print (keywords(str(text), ratio=SUMMARY_RATIO)[:99])

Summary (an online article using Gensim & web scraping using beautifulsoup):
wiki article title is:Deep learning - Wikipedia, Its text is:
Deep learning  (also known as deep structured learning) is part of a broader family of machine learn...
Wiki Summary:
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks,  convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]
The first general, working learning algorithm for supervised, deep, feedforward, multilayer perceptrons was published by Alexey Ivakhnenko and Lapa in 1967.[26] A 1971 paper described a deep network with eigh

In [ ]:
# BLEU SCORE and Rouge Score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

# allstr = '\n'.join(str(parser) for parser in summary)
# contents = '\n'.join(str(sentence1) for sentence1 in summary)

canidates = [(title+text).split()]
references = [sum.split()]
score = corpus_bleu(references, canidates)
print(f'BLEU SCORE:{score}')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score((title+text), sum)
print(f'ROUGE SCORE:{scores}')

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

BLEU SCORE:3.235229487165337e-232
ROUGE SCORE:{'rouge1': Score(precision=0.8105263157894737, recall=0.030771280138537364, fmeasure=0.059291581108829566), 'rougeL': Score(precision=0.4245614035087719, recall=0.016118289596376716, fmeasure=0.031057494866529773)}


## Summary of Extractive Summarization

 For extractive summarization:   
    -> Two out-of-box models look promising: Sumy’s LexRank and Gensim’s TextRank.  Try both for any specific domain/project first, and do check out others and do fine tune key hyperparameters (SENTENCES_COUNT), SUMMARY_RATIO)   
    -> Data sets for extractive summarization training:
        Opinosis dataset
        DUC 2002 dataset

# Abstractive Summarization:

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython import display 
from transformers import EncoderDecoderModel, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoModelWithLMHead
import warnings
warnings.filterwarnings('ignore')

## Abstractive Summarization - A Deep Learning Approach



### T5 Model (Text-To-Text Transfer Transformer)

- T5 model which was released by google research. It proposes **reframing ALL NLP tasks as an input-text-to-output-text formulation**  


**'Transfer Learning'**?   
- **"Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice..."**  
  




In [ ]:
def create_summary(model, tokenizer, text, max_length=512, min_length=30, length_penalty=5.0, repetition_penalty=5.0, num_beams=3):

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_length, truncation=True)
    outputs = model.generate(
        inputs, 
        max_length=max_length, 
        min_length=min_length, 
        length_penalty=length_penalty,
        repetition_penalty=repetition_penalty,
        num_beams=num_beams, 
        early_stopping=True)
    print(tokenizer.decode(outputs[0]))

### Test 1: News Summary
#### Load a news article random sample to summarize

In [ ]:
article = """ A typical example is a humanoid robot, the building of which has been regarded as the final target of robotics for many years, especially in Japan where many robotics researchers have been struggling with making humanoid robots work in our society. Regardless of such efforts, no definition of humanoid robots exists since “robot” itself has no clear definition. From the viewpoint of biology, human beings can be discriminated from other species by three distinctive features or capabilities. They are biped walking, use instrumentation, and have the invention/use of language. The first two capabilities have been attacked by robotics researchers as challenges in locomotion and manipulation that are the main issues in robotics. The third capability has not been considered as within robotics but linguistics. It seems far away from robotics. However, recent progress of research activities developed by the idea of “embodiment” in behavior-based robotics proposed by Rod Brooks at the MIT AI laboratory in the late 1980s has caused more conceptual issues such as body scheme, body image, self, consciousness, theory of mind, communication, and the emergence of language. Although these issues have been attacked in the existing disciplines such as brain science, neuroscience, cognitive science, and developmental psychology, robotics may project a new light on understanding these issues by constructing artifacts similar to us. Thus, robotics covers a broad range of disciplines; therefore it seems very difficult to find comprehensive textbooks to understand the area. Some textbooks focus on the fundamental issues on how to build robots while others focus on a limited area in robotics such as kinematics, dynamics, control, vision, or planning. Among them, Russell and Norvig's book from 1995 entitled Artificial Intelligence—A Modern Approach gave a good introduction to robotics from the viewpoint of AI. Pfeifer and Scheier's book from 1999 entitled Understanding Intelligence shows a constructive approach to understanding intelligence with a variety of robots from the viewpoint of embodied cognitive science. Readers can access resources on the recent activities of robotics around the world through the Web. """

#### T5-Base:

In [ ]:
!pip install Sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
sum = create_summary(model, tokenizer, article, 512, 120, 5.5, 5.5, 3)
print(sum)

<pad> robots can be discriminated from other species by three distinctive features or capabilities. the first two capabilities have been attacked by robotics researchers as challenges in locomotion. the third capability has not been considered as within robotics but linguistics. it seems far away from robotics, and may project a new light on understanding these issues. back to mail online home. back to the page you came from - http://www.cnn.com/reports/embeddedrobotics?v= (versiune<extra_id_1> gra<extra_id_20><extra_id_19></s>
None


#### t5-base-finetuned-summerize-news

In [ ]:
tokenizer_t5news = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model_t5news = AutoModelWithLMHead.from_pretrained('mrm8488/t5-base-finetuned-summarize-news')

print(create_summary(model, tokenizer, article, 512, 120, 5.5, 5.5, 3))

<pad> robots can be discriminated from other species by three distinctive features or capabilities. the first two capabilities have been attacked by robotics researchers as challenges in locomotion. the third capability has not been considered as within robotics but linguistics. it seems far away from robotics, and may project a new light on understanding these issues. back to mail online home. back to the page you came from - http://www.cnn.com/reports/embeddedrobotics?v= (versiune<extra_id_1> gra<extra_id_20><extra_id_19></s>
None


#### Observation  
- Above many 'n' padding can be cleaned up to produce a tidy result   
- It looks like for a specific NLP task (e.g. 'text summarization') for a specific domain (e.g. 'news summary' here), the fine-tuned model of news summarisation `t5-base-finetuned-summerize-news` produces a bit better and more concise summary than `T5-Base` model, thanks to the power of transfer learning using this specific news domain training data!  

### Test 2: Short Text Summary 
#### Load the same 'canary deployment' sample text for a like-for-like comparison with Extractive Summary

In [ ]:
# import os    
# from chardet import detect

# srcfile = "/content/sample_data/plain_text_sample_sec.txt"
# trgfile = "/content/sample_data/plain_text_sample_sec_ENC.txt"

# # get file encoding type
# def get_encoding_type(file):
#     with open(file, 'rb') as f:
#         rawdata = f.read()
#     return detect(rawdata)['encoding']

# from_codec = get_encoding_type(srcfile)

# # add try: except block for reliability
# try: 
#     with open(srcfile, 'r', encoding=from_codec) as f, open(trgfile, 'w', encoding='utf-8') as e:
#         text = f.read() # for small files, for big use chunks
#         e.write(text)

#     os.remove(srcfile) # remove old encoding file
#     os.rename(trgfile, srcfile) # rename new encoding
# except UnicodeDecodeError:
#     print('Decode Error')
# except UnicodeEncodeError:
#     print('Encode Error')

In [ ]:
with open('/content/sample_data/plain_text_sample_sec.txt', 'r') as f:
    text = f.read()
text

"Robotics, design, construction, and use of machines (robots) to perform tasks done traditionally by human beings. Robots are widely used in such industries as automobile manufacture to perform simple repetitive tasks, and in industries where work must be performed in environments hazardous to humans. Many aspects of robotics involve artificial intelligence; robots may be equipped with the equivalent of human senses such as vision, touch, and the ability to sense temperature. Some are even capable of simple decision making, and current robotics research is geared toward devising robots with a degree of self-sufficiency that will permit mobility and decision-making in an unstructured environment. Today’s industrial robots do not resemble human beings; a robot in human form is called an android.\nCompared to other artifacts with single functions, robots are expected to perform a variety of tasks, among which communication has an important role. A typical example is a humanoid robot, the 

#### T5-Base

In [ ]:
print(create_summary(model, tokenizer, text, 512, 120, 5.5, 5.5, 3))

<pad> many aspects of robotics involve artificial intelligence. robots may be equipped with the equivalent of human senses such as vision, touch, and the ability to sense temperature. today’s industrial robots do not resemble humans; a robot in human form is called an android. no definition of humanoid robots exists since “robot” itself has no clear definition. back to mail online home. back to the page you came from. click here for all the latest robotic news & updates. read our-rene<extra_id_1><extra_id_20></s>
None


#### t5-base-finetuned-summerize-news

In [ ]:
print(create_summary(model_t5news, tokenizer_t5news, text, 512, 120, 5.5, 5.5, 3))

<pad> robotics is the design, construction, and use of machines to perform tasks traditionally done by human beings. While many aspects of robotics involve artificial intelligence, they may be equipped with the equivalent of human senses such as vision, touch, and the ability to sense temperature. Humanoid robots are biped walking, use instrumentation, and have the invention/use of language. However, from the viewpoint of biology, humans can be discriminated from other species by three distinctive features or capabilities. Notably, this capability has not been considered within robotics but linguistics.</s>
None


#### Observation
It can be seen that the DL-enabled abstrative summary looks a bit **more natural, more coherent and more like human-writing summary** than the extractive summary! 

### Tips on how to produce an optimal text summary specific to your own text / domain? 

#### Option 1: Fine Tuning Text Summarization Hyperparameters
- As mentioned above, for specific NLP task of text summarization, you can always use those T5 models OOTB  
- But if you want to produce possibly the optimal summary result specific to your text data, one option is to fine tune the following T5 hyperprams  
- Finding the best values for many hyperprams is often **'dark arts'** in deep learning / artificial neural network. But one of good practices is **'trial and error'**: you can search and experiment until you find something satisfying. For example, if you want to do news summary, run search script as follows.  
- max_length
- min_length  
- length_penalty
- repetition_penalty

In [ ]:
model_t5news = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer_t5news = T5Tokenizer.from_pretrained("t5-base")

t5news_prams_dicts = {'penalty': [0.5, 1.0, 5.5, 10.5],
                     'max_length': [256,512,1024], 
                     'min_length': [60,120, 240] 
                     }

for length in t5news_prams_dicts['penalty']:
    for repetition in t5news_prams_dicts['penalty']:
        for max_l in t5news_prams_dicts['max_length']:
            for min_l in t5news_prams_dicts['min_length']: 
                %time summary = create_summary(model_t5news, tokenizer_t5news, article, max_l, min_l, length,repetition, 3)
                print(f"Experiment with length_penalty:{length}, repetition_penalty:{repetition}, max_length: {max_l}, min_length:{min_l}: {summary}")

<pad> no definition of humanoid robots exists since "robot" itself has no clear definition. human beings can be discriminated from other species by three distinctive features or capabilities. three capabilities have not been considered as within robotics but linguistics. humanoid robots have been a target of robotics research for many years.</s>
CPU times: user 9.87 s, sys: 370 ms, total: 10.2 s
Wall time: 10.2 s
Experiment with length_penalty:0.5, repetition_penalty:0.5, max_length: 256, min_length:60: None
<pad> no definition of humanoid robots exists since "robot" itself has no clear definition. human beings can be discriminated from other species by three distinctive features or capabilities. three capabilities have not been considered as within robotics but linguistics. humanoid robots have been a target of robotics research for many years. humanoid robots have been a target of many robotics researchers for years but no clear definition exists. a humanoid robot is a humano...</s>


#### Note
See: https://stackoverflow.com/questions/9879276/how-do-i-evaluate-a-text-summarization-tool) to quantify the performance.

### Option 2: Transfer Learning 

- This option is even better and fundamental solution.   
- For your specific domain problem and specific text data to summarize, we can and should use transfer learning and a small set of our own domain training data to train a custom T5 model optimal to our specific use case!  
- Basically you just need to load a pre-trained T5 model as a good starting point, and further train it with your domain data to fine tune it to better summarize your specific text  
- For example, if we'd like to summarize our meeting notes, as it is a different domain from news, we can collect a custom train dataset of (input text: raw meeting transcripts, output text (i.e. 'golden template'): meeting executive summary) 
https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization  

#### Create a clean summary by removing special tokens and spaces

In [ ]:
def clean_summarize(model, tokenizer, text, max_length=512, min_length=120, length_penalty=5.5, repetition_penalty=5.5, num_beams=3):
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    generated_ids = model.generate(
        input_ids=input_ids, 
        max_length=max_length,
        min_length=min_length, 
        repetition_penalty=repetition_penalty, 
        length_penalty=length_penalty, 
        num_beams=num_beams,
        early_stopping=True)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]

clean_summarize(model_t5news,tokenizer_t5news,text)


Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors


"humanoid robots. the third capability has not been considered as within robotics but linguistics. Thus, robotics covers a broad range of disciplines; therefore it seems very difficult to find comprehensive textbooks to understand the area. For example, Russell and Norvig's book from 1995 entitled Artificial Intelligence—A Modern Approach gave a good introduction to robotics from the viewpoint of AI. Pfeifer and Scheier's book from 1999 entitled Understanding Intelligence shows a constructive approach to understanding intelligenceversiune"

### Test 3: Long Text Summary 
- Using "Harry Potter and The Sorcerer's Stone" Story as a test sample

In [ ]:
with open('/content/sample_data/HarryPotter.txt', 'r') as f:
    long_text = f.read()
print(f'Harry Potter and The Sorcerer Stone length: {len(long_text)} characters\n{long_text[:200]}...')

Harry Potter and The Sorcerer Stone length: 3734 characters
Chapter Two
The Vanishing Glass

Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all. The sun rose on the sam...


#### T5-Base

In [ ]:
create_summary(model, tokenizer, long_text, 512, 120, 5.5, 5.5, 3)

<pad> ten years had passed since the Dursleys' nephew was born. his aunt, petunia, screeched on the door to wake him up. "up! get up! now!" said Harry; "get everything perfect on Duddy's birthday" dudley is still asleep at the moment but not for long. family says they haven't forgotten about their nephew in 10 years - and it's too late. read more of the book here: http://www.youtube.com/ (versiune<extra_id_1><extra_id_20><extra_id_19></s>


#### t5-base-finetuned-summerize-news

In [ ]:
clean_summarize(model_t5news,tokenizer_t5news,long_text)

'his aunt rapped on the door. "Up! Get up!" Harry groaned. "I want everything perfect on Duddy\'s birthday," said his aunt Petunia. "In the car crash when your parents died, and don\'t ask questions." Uncle Vernon entered the kitchen as Harry was turning over the bacon. "Comb your hair!" he barked, by way of a morning greeting. "You know what I mean?" He asked. "What did you say? You didn\'t tell me anything-'

#### long-t5-tglobal-base-16384 + BookSum
- This is a fine-tuned T5 model specialised in **summarizing long text** and get a SparkNotes-esque summary of arbitrary topics!  
- generalizes reasonably well to academic & narrative text  
- Reference: https://huggingface.co/pszemraj/

In [ ]:
# install/update Transformers 
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary"
)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
result = summarizer(long_text)
print(result[0]["summary_text"])

Ten years have passed since the Dudleys found their nephew on his front step. The sun rises and lights up the brass numbers four on the door, which is almost exactly as it had appeared on the night of Mr. Durson's first visit to the house. The photographs on the wall show how much time has passed. Now there are pictures of a young boy riding his bicycle with his father and playing a game with his mother. Harry is still there, sleeping at the moment. His aunt wakes him up and asks him to look after some bacon. He tells her that he wants everything perfect on dudley1s birthday. She says nothing but nothing. Harry gets dressed and goes down the hall to the kitchen. It looks like Dudly had gotten all the new things he wanted for his birthday without saying anything about the racing bike or the second television. In fact, only one thing makes sense to Harry: he doesn1t look any different from Dudle. He wears old clothes because of all those times Dudsley punched him in the nose. He has a th

In [ ]:
print(f'The original long text length: {len(long_text)} chars\nAbstractive summary length: {len(result[0]["summary_text"])} chars')

The original long text length: 3734 chars
Abstractive summary length: 1169 chars


#### Summary
- Again here we see the fine-tuned T5 model specialised in summarizing long text gives better performance than the base T5 model or the model fined tuned for other domains such as news  
- **'No one size fits all.'** in data science we always have to train and fine tune a model to achieve the best possible performnace in a specific domain before 
Artificial General Intelligence (AGI) is coming, which is another topic to discuss later  
- For text summarization, DL-based abstractive summary techniques often achieve SOTA performance instead of traditional extractive summary techniques  
- Thus if you are not sure to use which text summarization tool for your specific text data, first try those OOB abstractive summarization models and you have a good chance of obtaining good results with speed and accuracy!